# Modelling herb runs in Old School Runescape

## 02-Theoretical Framework
If we define the extra amount of herbs harvested on a single herb patch as a random variable $X$ say, then we can consider modelling it as a negative binomial random variable. As a player continues harvesting extra herbs until the first failure. Each individual harvest of a herb can be treated as a Bernoulli trial with some unknown fixed probability of failure $p$ say. This probability may have different dependenies such as: what type of herb is planted, what the players farming level is, and which of the 9 herb patches the herbs are being harvested from. Hence, this problem requires some sort of method to solve for this unknown probability $p$ on each herb patch. Once found we can construct the particular negative binomial distribution in question and do some exploratory simulation.

We can use the collected data to obtain a maximum likelihood estimate (MLE) of $p$. This corresponds to a very good unbiased estimate based on the collected data which has many useful properties that we want in an estimator (read more on MLEs in this great [article](https://towardsdatascience.com/probability-concepts-explained-maximum-likelihood-estimation-c7b4342fdbb1)). To do so we need to contruct a likelihood function from the known probability mass function of a negative binomial random variable. In general if we let $Y\sim {NB}(1,p)$ then as listed [here](https://en.wikipedia.org/wiki/Negative_binomial_distribution), the probability mass function (PMF) is
$$P(Y=y) = \binom{y+1-1}{y} p^{y}(1-p)^{1} = p^y(1-p).$$

Now we need to consider switching to a likelihood based perspective. If we let $\theta$ be a guess of $p$ based on the collected data labelled $Y_1=y_1, Y_2=y_2,\dots Y_n=y_n$ for $n$ observations of extra herbs harvested at one particular patch. Then the probability that the chosen $\theta$ fits an individual observation is the same as above now with fixed $y_i$ (for $i$ labelling each data-point) in place of fixed $p$
$$l(\theta\ \vert \ Y_i=y_i) = P_{\theta} (Y_i=y_i) = \theta^{y_i}(1-\theta).$$

Which gives the joint probability that the chosen $\theta$ fits all of the data as the product of all of these individual PMFs (as each number of extra herbs harvested at each patch is independent and identically distributed)
$$L\left(\theta \ \vert \ Y_1=y_1, Y_2=y_2,\dots Y_n=y_n\right) = \prod_{i=1}^n \theta^{y_i}(1-\theta).$$

To find the MLE based on our given data we need to maximise the above function in terms of choice of $\theta$. Maximising such a function is quite difficult so we can consider using a log transform to transform the above from a product to a summation. This will have no effect on the $\theta$ that maximises $L$. We compute the log likelihood by
$$\begin{aligned} 
\mathcal{L}\left(\theta \ \vert \ Y_1=y_1, Y_2=y_2,\dots Y_n=y_n\right) &= \log\{L\left(\theta \ \vert \ Y_1=y_1, Y_2=y_2,\dots Y_n=y_n\right)\} \\
&= \log\left\{\prod_{i=1}^n \theta^{y_i}(1-\theta)\right\} \\
&= \sum_{i=1}^n \log\left\{\theta^{y_i}(1-\theta)\right\} \\
&= \sum_{i=1}^n \left[y_i\log(\theta) + \log(1-\theta)\right] \\
&= \log(\theta)\sum_{i=1}^n y_i + n\log(1-\theta).
\end{aligned}$$

We could then consider maximising this log likelihood by the first and second derivative tests or numerically using Python's SciPy package, which inclues an optimisation kit (scipy.optimize). I have chosen to do both so that I can compare the accuracy of the numerical method with the analytic solution. 

To find extrema of the above function we take the first derivative and set it to zero such that
$$\begin{aligned}
\frac{dL}{d\theta} &= 0 \\
\frac{d}{d\theta}\left[\log(\theta)\sum_{i=1}^n y_i + n\log(1-\theta)\right] &= 0 \\
\frac{1}{\theta}\sum_{i=1}^n y_i - \frac{n}{1-\theta} &= 0 \\
\frac{1}{\theta}\sum_{i=1}^n y_i &= \frac{n}{1-\theta} \\
(1-\theta)\sum_{i=1}^n y_i &= n\theta \\
\sum_{i=1}^n y_i - \theta \sum_{i=1}^n y_i &= n\theta \\
\sum_{i=1}^n y_i &= n\theta + \theta \sum_{i=1}^n y_i \\
\theta \left(n + \sum_{i=1}^n y_i\right) &= \sum_{i=1}^n y_i \\
\theta &= \frac{\sum_{i=1}^n y_i}{n + \sum_{i=1}^n y_i}.
\end{aligned}$$

Now for the sake of clarity consider labelling this extrema as $\vartheta$, we can divide both numerator and denominator of the resulting fraction by $n$ to find a nice expression just in terms of the mean of the data $\bar{Y}$
$$\vartheta = \frac{\bar{Y}}{1+\bar{Y}}.$$

Next we need to check the boundary values that $\theta$ can take as well, as these could also possibly be extrema. As $\theta$ is a probability it is bounded by $0$ and $1$. Hence, the set of possible extrema for this function is $\{0,\vartheta,1\}$. Next we need to check if each extrema is a maximum or a minimum of the function by the second derivative test. The extrema that is the maximum will be the MLE
$$\begin{aligned}
\frac{d^2L}{d\theta^2} &= \frac{d}{d\theta}\left[\frac{dL}{d\theta}\right] \\
&= \frac{d}{d\theta}\left[\frac{1}{\theta}\sum_{i=1}^n y_i - \frac{n}{1-\theta}\right] \\
&= -\frac{1}{\theta^2}\sum_{i=1}^n y_i - \frac{n}{(1-\theta)^2}.
\end{aligned}$$

Now we can note that this second derivative is undefined at both $\theta = 0$ and $\theta = 1$ due to division by zero. Hence, these values cannot be extrema. That leaves just $0<\vartheta<1$ to be checked. We can see that for any $\vartheta$ in its specified range that the second derivative will be negative. So by the second derivative test we find this extrema to be a maximum. Hence, the MLE of $p$
$$\text{MLE}(p) = \frac{\bar{Y}}{1+\bar{Y}}.$$

We can use this MLE as our estimate of the true probability of failing to pick a herb at a particular patch $p$.
